In [17]:
import torch
import pickle
def load_inputs_from_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

# 从文件中读取数据
loaded_inputs1 = load_inputs_from_file('inputs.pkl')
loaded_inputs2 = load_inputs_from_file('true.pkl')


In [18]:
print(loaded_inputs1['input_ids'][0])
print(loaded_inputs2['input_ids'][0])

tensor([    1,     0,     0,  ...,  1009,  4423, 29973], device='cuda:0')
tensor([    1,     0,     0,  ...,  1009,  4423, 29973], device='cuda:0')


In [ ]:
from transformers import AutoModelForCausalLM, LlamaTokenizer
TOKENIZER_PATH = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/modle_weight/vicuna-7b-v1.5"
tokenizer = LlamaTokenizer.from_pretrained(TOKENIZER_PATH)
id = tokenizer.encode([' '])
print(id)


/opt/conda/envs/cogvlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/cogvlm/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[1, 0]


In [11]:
response1 = tokenizer.decode(loaded_inputs1['input_ids'][0])
response2 = tokenizer.decode(loaded_inputs2['input_ids'][0])
print(response1)
print(response2)

<s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><u

In [16]:
response = tokenizer.decode('29871')
print(response)

</s><s>


In [25]:
import re
def parse_bbox_from_response(response):
    """
    从模型响应中解析边界框坐标
    """
    # 寻找[x, y, x2, y2]格式的坐标，包括整数和小数
    pattern = r'\[\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]\]'
    match = re.search(pattern, response)
    
    if match:
        try:
            x1 = int(match.group(1))  # 提取整数值
            y1 = int(match.group(2))
            x2 = int(match.group(3))
            y2 = int(match.group(4))
            # 验证坐标有效性，如果仍需要确保有效性范围
            if x1 <= x2 and y1 <= y2:
                x1 = x1/1000
                x2 = x2/1000
                y1 = y1/1000
                y2 = y2/1000
                return [x1, y1, x2, y2]
        except ValueError:
            pass
    
    # 不符合条件的返回None
    return None

In [26]:
response = "[[270,159,624,975]]"
bbox = parse_bbox_from_response(response)
print(bbox)

[0.27, 0.159, 0.624, 0.975]


In [4]:
import json
file1 = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/fgeo_1_10_bbox.json'
file2 = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/filter_geoqa+_1_10.json'
# 读取两个 JSON 文件
with open(file1, 'r') as f:
    data1 = json.load(f)

with open(file2, 'r') as f:
    data2 = json.load(f)

# 存储不满足条件的样本 ID
mismatched_ids = []

# 遍历第一个 JSON 文件中的每个记录
for item1 in data1:
    id1 = item1['id']
    
    # 在第二个 JSON 文件中找到匹配的记录
    item2 = next((item for item in data2 if item["id"] == id1), None)
    if item2 is None:
        continue

    # 检查第二个文件的 conversations 长度是否为第一个文件的两倍
    if len(item2['conversations']) != 2 * len(item1['conversations']):
        mismatched_ids.append(id1)

# 打印不满足条件的样本 ID
print("不满足条件的样本ID：", mismatched_ids)

不满足条件的样本ID： []


In [ ]:
# 过滤掉整张图片的描述
import json
def calculate_iou(box1, box2):
    """Calculate the Intersection over Union (IoU) of two bounding boxes."""
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

file_path = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/fgeo_1_10_bbox.json'
with open(file_path, 'r') as f:
    data1 = json.load(f)

reference_box = [0, 0, 1, 1]  # The reference box to compare against
for sample_data in data1:
    for conversation in sample_data["conversations"]:
        for attr in ["<CAPTION>", "<REASONING>"]:
            if conversation[attr] is not None:
                for item in conversation[attr]:
                    iou = calculate_iou(item["bbox"], reference_box)
                    if iou > 0.8:
                        item["bbox"] = None

# Output the modified sample_data
with open('/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/test.json', 'w') as f:
    json.dump(data1, f, indent=2)

In [ ]:
# 统计有效bbox个数
file_path = '/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/test.json'
with open(file_path, 'r') as f:
    data1 = json.load(f)

count = 0
for sample_data in data1:
    for conversation in sample_data["conversations"]:
        for attr in ["<CAPTION>", "<REASONING>"]:
            if conversation[attr] is not None:
                for item in conversation[attr]:
                    if item['bbox'] is not None:
                        count += 1
print(f"The number of bounding boxes that are not None: {count}")

The number of bounding boxes that are not None: 48


In [ ]:
# 将bbox放回原文本
import json
import re

# 读取两个 JSON 文件
with open(file1, 'r') as f:
    data1 = json.load(f)

with open(file2, 'r') as f:
    data2 = json.load(f)

# 遍历第一个 JSON 文件中的每个记录
for item1 in data1:
    id1 = item1['id']
    
    # 在第二个 JSON 文件中找到匹配的记录
    item2 = next((item for item in data2 if item["id"] == id1), None)
    if item2 is None:
        continue

    # 遍历第一个记录中的 conversations，同时与第二个文件的对话对应
    for idx, conversation1 in enumerate(item1['conversations']):
        if idx * 2 + 1 >= len(item2['conversations']):
            break
        
        # 获取对应的 from gpt 文本
        gpt_response = item2['conversations'][idx * 2 + 1]['value']
        
        # 处理 <CAPTION>
        if "<CAPTION>" in conversation1 and conversation1["<CAPTION>"] is not None:
            caption_phrases = {phrase["content"]: phrase["bbox"] for phrase in conversation1["<CAPTION>"] if phrase["bbox"] is not None}
            caption_content = re.search(r'<CAPTION>(.*?)</CAPTION>', gpt_response, re.DOTALL)
            if caption_content:
                updated_caption = caption_content.group(1)
                for phrase, bbox in caption_phrases.items():
                    if phrase in updated_caption:
                        updated_caption = updated_caption.replace(phrase, f"{phrase} {bbox}")
                gpt_response = gpt_response.replace(caption_content.group(1), updated_caption)
        
        # 处理 <REASONING>
        if "<REASONING>" in conversation1 and conversation1["<REASONING>"] is not None:
            reasoning_phrases = {phrase["content"]: phrase["bbox"] for phrase in conversation1["<REASONING>"] if phrase["bbox"] is not None}
            reasoning_content = re.search(r'<REASONING>(.*?)</REASONING>', gpt_response, re.DOTALL)
            if reasoning_content:
                updated_reasoning = reasoning_content.group(1)
                for phrase, bbox in reasoning_phrases.items():
                    if phrase in updated_reasoning:
                        updated_reasoning = updated_reasoning.replace(phrase, f"{phrase} {bbox}")
                gpt_response = gpt_response.replace(reasoning_content.group(1), updated_reasoning)

        # 更新 gpt_response
        item2['conversations'][idx * 2 + 1]['value'] = gpt_response

# 输出处理后的 JSON 数据
with open('/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/test.json', 'w') as f:
    json.dump(data2, f, indent=2)

In [ ]:
import os
import json
import torch
import argparse
from PIL import Image
import datetime
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from transformers import AutoModelForCausalLM, LlamaTokenizer
from tqdm import tqdm
import re
TOKENIZER_PATH = "/lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/model/vicuna-7b-v1.5"

def custom_collate_fn(batch):
    """
    自定义的collate函数，用于处理PIL.Image对象
    """
    id_list = [item['id'] for item in batch]
    image_list = [item['image'] for item in batch]
    conversations_list = [item['conversations'] for item in batch]
    original_image_path_list = [item['original_image_path'] for item in batch]
    
    return {
        'id': id_list,
        'image': image_list,
        'conversations': conversations_list,
        'original_image_path': original_image_path_list

    }
class BboxDataset(Dataset):
    """
    用于加载预处理好的图像和标注数据的数据集
    """
    def __init__(self, json_file, image_base_dir):
        """
        Args:
            json_file (str): 包含图像路径和标注的JSON文件路径
            image_base_dir (str): 图像文件的基础目录
        """
        with open(json_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.image_base_dir = image_base_dir
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_base_dir, item['image'])
        
        # 加载图像
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # 返回一个占位图像
            image = Image.new('RGB', (224, 224), color='gray')
        
        conversations = item['conversations']
        # 深度复制会话数据，防止在处理过程中修改原始数据
        # conversations = []
        # for conv in item['conversations']:
        #     conversation = {}
        #     for key, value in conv.items():
        #         conversation[key] = value.copy() if isinstance(value, list) else value
        #     conversations.append(conversation)
            
        return {
            'id': item['id'],
            'image': image,
            'conversations': conversations,
            'original_image_path': item['image']
        }


def parse_bbox_from_response(response):
    """
    从模型响应中解析边界框坐标
    """
    # 寻找[x, y, x2, y2]格式的坐标，包括整数和小数
    pattern = r'\[\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]\]'
    match = re.search(pattern, response)
    
    if match:
        try:
            x1 = int(match.group(1))  # 提取整数值
            y1 = int(match.group(2))
            x2 = int(match.group(3))
            y2 = int(match.group(4))
            # 验证坐标有效性，如果仍需要确保有效性范围
            if x1 <= x2 and y1 <= y2:
                x1 = x1/1000
                x2 = x2/1000
                y1 = y1/1000
                y2 = y2/1000
                return [x1, y1, x2, y2]
        except ValueError:
            pass
    
    # 不符合条件的返回None
    return None
def bbox_convert(bbox):
    # 将每个元素乘以1000
    scaled_bbox = [coord * 1000 for coord in bbox]
    return f"[{scaled_bbox}]"


def generate_bbox_prompt(bbox, content):
    """
    为列表元素生成提示，用于请求模型生成边界框
    """
    return (f"Tell me if the designated area {bbox} in the picture is '{content}', yes or no?")


def setup_distributed(rank, world_size):
    """
    设置分布式训练环境
    """
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    
    # 初始化进程组
    dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=datetime.timedelta(seconds=3600))
    
    # 设置GPU设备
    torch.cuda.set_device(rank)


def cleanup_distributed():
    """
    清理分布式训练环境
    """
    dist.destroy_process_group()

/opt/conda/envs/cogvlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/cogvlm/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
data_json = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k/json/filter_entire_image_caption.json"
image_dir = "/mnt/pfs-mc0p4k/nlu/team/yuhaofu/data/LLaVA-CoT-100k"
dataset = BboxDataset(data_json, image_dir)
# sampler = DistributedSampler(
#     dataset, 
#     num_replicas=world_size, 
#     rank=rank, 
#     shuffle=False
# )

dataloader = DataLoader(
    dataset, 
    batch_size=1,  # 设置为1，每次处理一个样本
    # sampler=sampler,
    num_workers=1,
    collate_fn=custom_collate_fn
)

In [6]:
for batch in dataloader:
    sample_id = batch['id'][0]
    image = batch['image'][0]
    conversations = batch['conversations'][0]  # 一批中的第一个样本
    image_path = batch['original_image_path'][0]

    # image_path = os.path.join(args.image_dir, image_path)
    # image = Image.open(image_path).convert('RGB')
    processed_conversations = []
    for conversation in conversations:
        processed_conv = {}
        # print(conversation)
        # 处理 <CAPTION> 列表
        # def process_elements(elements, model, tokenizer, image, rank, args):
        #     results_with_bbox = []

        #     for element in elements:
        #         if element['bbox'] is not None:
        #             bbox = element['bbox']
        #             content = element['content']

        #             convert_bbox = bbox_convert(bbox)
        #             prompt = generate_bbox_prompt(convert_bbox, content)

        #             input_by_model = model.module.build_conversation_input_ids(
        #                 tokenizer, 
        #                 query=prompt,
        #                 history=None, 
        #                 images=[image]
        #             )
                    
        #             inputs = {
        #                 'input_ids': input_by_model['input_ids'].unsqueeze(0).to(rank),
        #                 'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(rank),
        #                 'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(rank),
        #                 'images': [[input_by_model['images'][0].to(rank).to(torch.bfloat16)]] if image is not None else None,
        #             }
                    
        #             if 'cross_images' in input_by_model and input_by_model['cross_images']:
        #                 inputs['cross_images'] = [[input_by_model['cross_images'][0].to(rank).to(torch.bfloat16)]]
                    
        #             # 生成参数
        #             gen_kwargs = {
        #                 "max_length": args.max_tokens,
        #                 "do_sample": False
        #             }
                    
        #             # 生成响应
        #             outputs = model.module.generate(**inputs, **gen_kwargs)
        #             outputs = outputs[:, inputs['input_ids'].shape[1]:]
        #             response = tokenizer.decode(outputs[0])
        #             # print('image_path:', image_path, 'prompt:', prompt)
        #             # print('response:', response)
        #             response = response.split("</s>")[0]
        #             print(response)
        #             flag = 1
        #             if 'yes' in response.lower():
        #                 bbox = bbox
        #             elif 'no' in response.lower():
        #                 bbox = element['bbox']
        #             else:
        #                 flag = 0
        #         else:
        #             bbox = element['bbox']
        #             flag = 1
        #         results_with_bbox.append({
        #             "content": element['content'],
        #             "bbox": bbox,
        #             "flag": flag
        #         })
        #         print(flag)
            
        #     return results_with_bbox
        if "<CAPTION>" in conversation and conversation["<CAPTION>"]:
            caption_elements = conversation["<CAPTION>"]
            for element in caption_elements:
                if element['bbox'] is not None:
                    bbox = element['bbox']
                    content = element['content']
                    print(bbox)
                    convert_bbox = bbox_convert(bbox)
                    prompt = generate_bbox_prompt(convert_bbox, content)
                    # print(convert_bbox)
                    # print(prompt)
            # processed_conv["<CAPTION>"] = process_elements(
            #     caption_elements, model, tokenizer, image, rank, args
            # )
        else:
            processed_conv["<CAPTION>"] = None
        break
    

        processed_conversations.append(processed_conv)
    break

Error loading image /lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/coco/train2017/000000338745.jpg: [Errno 2] No such file or directory: '/lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/coco/train2017/000000338745.jpg'
Error loading image /lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/pisc/image/09543.jpg: [Errno 2] No such file or directory: '/lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/pisc/image/09543.jpg'
Error loading image /lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/coco/train2017/000000440900.jpg: [Errno 2] No such file or directory: '/lpai/volumes/ss-nlu-ali-sh/lizr/yuhaofu/data/LLaVa-CoT-100k/coco/train2017/000000440900.jpg'
[0.27, 0.159, 0.624, 0.975]
[0.0, 0.711, 0.35, 0.993]
[0.27, 0.159, 0.624, 0.975]
